In [2]:
import requests
from openai import OpenAI
import os
import json
import re
from tqdm import tqdm
from datasets import load_dataset

client = OpenAI(api_key = '')

In [3]:
dataset = load_dataset('huseinzol05/malaysian-dialect-qa', split = 'test')

In [ ]:
dataset_lang = load_dataset('huseinzol05/malaysian-dialect-qa-lang', split = 'test')

In [4]:
questions = []
for i in range(len(dataset)):
    q = dataset[i]['question'] + '\n\nAfter that, put your final answer within $\\boxed{}$.'
    questions.append((i, q))
    
len(questions)

140

In [5]:
questions[0][1]

(0,
 'Lepas ujan jangan maen lari-lari, kan biyak.\n\nterjemah ke melayu baku\n\nAfter that, put your final answer within $\\boxed{}$.')

In [18]:
folder = 'openai-chatgpt4.1'
# !rm -rf {folder}
!mkdir {folder}

mkdir: cannot create directory ‘openai-chatgpt4.1’: File exists


In [22]:
def generate_answer(row, repeat = 5):
    no, q = row
    for k in range(repeat):
        filename = os.path.join(folder, f'{no}-{k}.json')
        if os.path.exists(filename):
            continue
        
        for _ in range(5):

            completion = client.chat.completions.create(
                model="gpt-4.1",
                messages=[
                    {
                        "role": "user",
                        "content": questions[i][1]
                    }
                ]
            )
            
            text = completion.choices[0].message.content
            try:
                if 'boxed{' in text:
                    text = text.split('boxed{')
                    text = text[-1].split('}')[0]
                    if 'text{' in text:
                        text = text.split('text{')[1]
                with open(filename, 'w') as fopen:
                    json.dump(text, fopen)
                break
            except Exception as e:
                print(e)

In [24]:
for i in tqdm(range(len(dataset))):
    generate_answer(questions[i])

100%|█████████████████████████████████████████| 140/140 [31:35<00:00, 13.54s/it]


In [43]:
from sacrebleu.metrics import CHRF
from glob import glob
from collections import defaultdict
import numpy as np

In [39]:
chrf = CHRF()
pairs = defaultdict(list)

for i in tqdm(range(len(dataset_lang))):
    from_lang = dataset_lang[i]['from_lang']
    to_lang = dataset_lang[i]['to_lang']
    gt = dataset_lang[i]['answer']
    pair = f'{from_lang}<>{to_lang}'
    files = glob(f'openai-chatgpt4.1/{i}-*.json')
    scores = []
    for f in files:
        with open(f) as fopen:
            d = json.load(fopen)
        score = chrf.corpus_score([d], [[gt]]).score
        scores.append(score)

    max_score = max(scores)
    pairs[pair].append(max_score)

100%|███████████████████████████████████████| 140/140 [00:00<00:00, 1063.98it/s]


In [47]:
for k, v in pairs.items():
    l, r = k.split('<>')
    print(f'From: {l} To: {r}, score:', np.mean(v))

From: johor To: malay, score: 54.78567833962654
From: kedah To: malay, score: 61.12657281892719
From: pahang To: malay, score: 58.956758947267325
From: negeri sembilan To: malay, score: 58.276749455174375
From: kelantan To: malay, score: 48.246166249336184
From: penang To: malay, score: 68.75697140481608
From: melaka To: malay, score: 48.636754195346946
From: malay To: johor, score: 51.79138167993752
From: malay To: kedah, score: 53.0860308519873
From: malay To: pahang, score: 45.08805800936419
From: malay To: negeri sembilan, score: 45.7387086778156
From: malay To: kelantan, score: 38.05722296237036
From: malay To: penang, score: 57.909930402335206
From: malay To: melaka, score: 55.77760894766224


In [53]:
x = """
From: malay To: johor, score: 51.79138167993752
From: malay To: kedah, score: 53.0860308519873
From: malay To: pahang, score: 45.08805800936419
From: malay To: negeri sembilan, score: 45.7387086778156
From: malay To: kelantan, score: 38.05722296237036
From: malay To: penang, score: 57.909930402335206
From: malay To: melaka, score: 55.77760894766224
"""

scores = []
for l in x.split('\n'):
    if 'score:' not in l:
        continue
    
    scores.append(float(l.split('score: ')[1]))
    
np.mean(scores)

49.63556307592463